In [ ]:
using DrWatson
@quickactivate "JEG"

using ReinforcementLearning
using IntervalSets
using LinearAlgebra
using ControlSystems
using CUDA
using PlotlyJS


include(srcdir("node_constructor.jl"))
include(srcdir("electric_grid_env.jl"));
include(srcdir("sin_policy.jl"))
include(srcdir("data_hook.jl"))
include(srcdir("solar_module.jl"))

# #17 Check/Test Notebook
To be transfered to unit tests (-> https://erik-engheim.medium.com/julia-v1-5-testing-best-practices-3ca8780e6336)?
Reconstruction of parameter dict, checking if 'all' cases run through

In [ ]:
CM = [ 0. 0. 1.
     0. 0. 2
     -1. -2. 0.]

ts = 1e-4

Test setting without parameter dict and CM

In [ ]:
env = ElectricGridEnv(ts=ts, use_gpu=false, num_sources = 20, num_loads = 2, maxsteps = 500);

In [ ]:
env.nc.parameters#["loads"]

### Source Check
Test with CM and only one defined source + grid-params one left out

In [ ]:
parameters = Dict()
source_list = []
source1 = Dict()
source2 = Dict()

# all possible parameters: (only part of them defined)
#source["pwr"] = 45000.0
#source["v_rip"] = 0.01556109320329396
#source["vdc"] = 750
#source["i_rip"] = 0.10108821490394984
source1["fltr"] = "LCL"
source1["source_type"] = "pv"
source1["R1"] = 0.4
source1["R_C"] = 0.0006
#source1["L1"] = 2.3e-3
#source["R2"] = 0.4022094955070556   # needed for LCL
#source["L2"] = 0.001005523738767639
#source1["C"] = 1e-6;


push!(source_list, source1)

parameters["source"] = source_list
parameters["grid"] = Dict("phase" => 1, "v_rms" => 230);

In [ ]:


ts = 1e-4
env = ElectricGridEnv(ts=ts, CM = CM, num_sources = 2, num_loads = 1, parameters = parameters, maxsteps = 500)
env.nc.parameters

In [ ]:
env.nc.parameters["source"][1]

In [ ]:
env.nc.parameters["source"][2]

In [ ]:
env.v_dc_arr[1](env, 800, 25)

In [ ]:
env.v_dc_arr[2](env, 1000, 25)

### Cable check
Only parts of the sorces defined, rest should be layouted depending on peak power - btw. other way round does NOT fit (L -> pwr, if pwr not defined)
Only one cable defined party, rest filled up and other calbes put to dict?

In [ ]:
parameters = Dict()
source_list = []
source1 = Dict()
source2 = Dict()

source1["pwr"] = 5000.0
source1["vdc"] = 750
source1["fltr"] = "LC"

source2["pwr"] = 5000.0
source2["vdc"] = 750
source2["fltr"] = "L"

push!(source_list, source1, source2);

load_list = []
load = Dict()

load["impedance"] = "RLC"
load["R"] = 14.0;
load["L"] = 57.042;
load["C"] = 39.18;
push!(load_list, load);

cable_list = []

cable = Dict()
cable["R"] = 0.722
cable["L"] = 0.264e-3
push!(cable_list, cable);

parameters["source"] = source_list
parameters["cable"] = cable_list
parameters["load"] = load_list;

In [ ]:
ts = 1e-4
env = ElectricGridEnv(ts=ts, CM = CM, num_sources = 2, num_loads = 1, parameters = parameters, maxsteps = 500)
env.nc.parameters

In [ ]:
env.nc.parameters["cable"][1]

In [ ]:
env.nc.parameters["cable"][2]

### Load Check

In [ ]:
load_list = []
load = Dict()

load["impedance"] = "RLC"
load["R"] = 14.0;
load["L"] = 57.042;
#load["C"] = 39.18;
push!(load_list, load);


parameters["load"] = load_list;

In [ ]:
ts = 1e-4
env = ElectricGridEnv(ts=ts, CM = CM, num_sources = 2, num_loads = 1, parameters = parameters, maxsteps = 500)
env.nc.parameters

In [ ]:
env.nc.parameters["load"][1]

In [ ]:
env.nc.parameters["load"][1]["C"]

In [ ]:
load_list = []
load = Dict()

#load["impedance"] = "RLC"
load["R"] = 14.0;
#load["L"] = 57.042;
#load["C"] = 39.18;
push!(load_list, load);


parameters["load"] = load_list;

ts = 1e-4
env = ElectricGridEnv(ts=ts, CM = CM, num_sources = 2, num_loads = 1, parameters = parameters, maxsteps = 500)
env.nc.parameters["load"][1]

In [ ]:
load_list = []
load = Dict()

#load["impedance"] = "RLC"
load["R"] = 14.0;
#load["L"] = 57.042;
#load["C"] = 39.18;
push!(load_list, load);


parameters["load"] = load_list;

ts = 1e-4
env = ElectricGridEnv(ts=ts, CM = CM, num_sources = 2, num_loads = 1, parameters = parameters, maxsteps = 500)
env.nc.parameters["load"][1]

--------------------------------------------------------------------

# TODO: LC oder L vor LCL: Problem, zusammenbasteln der A-Matrix

Das folgende geht kaputt, weil der NC annimmt, dass zuerst immer LCL, dann LC und dann L Filter kommen. Noch richtig schön viel zu fuckeln, jiha

In [ ]:
parameters = Dict()
source_list = []
source1 = Dict()
source2 = Dict()

source1["pwr"] = 5000.0
source1["vdc"] = 750
source1["fltr"] = "LC"

source2["pwr"] = 5000.0
source2["vdc"] = 750
source2["fltr"] = "LCL"

push!(source_list, source1, source2);

parameters["source"] = source_list


env = ElectricGridEnv(ts=ts, CM = CM, num_sources = 2, num_loads = 1, parameters = parameters, maxsteps = 500)
env.nc.parameters